Load packages

In [1]:
#loading packages
push!(LOAD_PATH, "src");
import Base: @kwdef
using Test
using LifeInsuranceDataModel

using BitemporalPostgres
using SearchLight
using SearchLightPostgreSQL
using TimeZones
using ToStruct
using JSON

In [ ]:
purging the data model entirely - empty the schema

In [ ]:
if (haskey(ENV, "GENIE_ENV") && ENV["GENIE_ENV"] == "dev")
    if (haskey(ENV, "GITPOD_REPO_ROOT"))
        run(```psql -f sqlsnippets/droptables.sql```)
    else
        println("execute this in Your REPL")run(```sudo -u postgres psql -f sqlsnippets/droptables.sql```)
        println("""""")
    end
end

Loading the data model- Create tables, constraints etc. and load Roles

loading inverses of the role tables to provide role descriptions in object creation,  for instance like in: "ref_role=cpRole["Policy Holder"]

In [ ]:
LifeInsuranceDataModel.load_model()

cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end


Or just connect to an existing model

In [3]:
LifeInsuranceDataModel.connect()

PostgreSQL connection (CONNECTION_OK) with parameters:
  user = postgres
  password = ********************
  channel_binding = prefer
  dbname = postgres
  host = 127.0.0.1
  port = 5432
  client_encoding = UTF8
  options = -c DateStyle=ISO,YMD -c IntervalStyle=iso_8601 -c TimeZone=UTC
  application_name = LibPQ.jl
  sslmode = prefer
  sslcompression = 0
  sslsni = 1
  ssl_min_protocol_version = TLSv1.2
  gssencmode = prefer
  krbsrvname = postgres
  target_session_attrs = any

Create a Partner

In [4]:
# create Partner
p = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="Partner 1")
w = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w)
create_component!(p, pr, w)
commit_workflow!(w)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 4 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'4' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 4 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed" ) VALUES ( E'4', E'4', E'2038-01-19T03:14:06.999+00:00', E'2014-05-30T21:00:01.001+01:00', 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_history" AS "workflows_ref_history", "workflows"."ref_version" AS "workflows_ref_version", "workflows"."tsdb_validfrom" AS "workflows_tsdb_validfrom", "workflows"."tsw_validfrom" AS "workflows_tsw_validfrom", "workflows"."is_committed" AS "workflows_is_committed" FROM "workflows" WHERE "id" = 4 ORDER BY workflows.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: INSERT INTO validityintervals ( "ref_history", "ref_version", "tsdb_validfrom", "tsworld_validfrom", "tsdb_invalidfrom", "tsworld_invalidfrom", "is_committed" ) VALUES ( E'4', E'4', E'2022-07-08T07:32:25.038+01:00', E'2014-05-30T20:00:01.001+00:00', E'2038-01-19T03:14:06.999+00:00', E'2038-01-19T03:14:06.999+00:00', 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
NOTICE:  NEW: (4,4,4,"2014-05-30 20:00:01.001+00","2038-01-19 03:14:06.999+00","2022-07-08 06:32:25.038+00","2038-01-19 03:14:06.999+00",,,0)
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom"

...

┌ Info: INSERT INTO partners ( "ref_history", "ref_version" ) VALUES ( E'4', E'4' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "partners"."id" AS "partners_id", "partners"."ref_history" AS "partners_ref_history", "partners"."ref_version" AS "partners_ref_version" FROM "partners" WHERE "id" = 2 ORDER BY partners.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: INSERT INTO partnerrevisions ( "ref_component", "ref_validfrom", "ref_invalidfrom", "description" ) VALUES ( E'2', E'4', E'9223372036854775807', E'Partner 1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
NOTICE:  NEW: (2,2,4,9223372036854775807,,"Partner 1")
┌ Info: SELECT "partnerrevisions"."id" AS "partnerrevisions_id", "partnerrevisions"."ref_component" AS "partnerrevisions_ref_component", "partnerrevisions"."ref_validfrom" AS "partnerrevisions_ref_validfrom", "partnerrevisions"."ref_invalidfrom" AS "partnerrevisions_ref_invalidfrom", "partnerrevisions"."description" AS "partnerrevisions_description" FROM "partnerrevisions" WHERE "id" = 2 ORDER BY partnerrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'4' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validi

┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history = E'4' AND tsrdb @> TIMESTAMPTZ E'2038-01-18T03:14:06.999+00:00' AND tsrworld && tstzrange(E'2014-05-31T20:00:01.001+00:00',E'2038-01-19T03:14:06.999+00:00') AND is_committed=1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: UPDATE validityintervals

In [5]:
find(Partner)

┌ Info: SELECT "partners"."id" AS "partners_id", "partners"."ref_history" AS "partners_ref_history", "partners"."ref_version" AS "partners_ref_version" FROM "partners" ORDER BY partners.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


2-element Vector{Partner}:
 Partner
| KEY               | VALUE |
|-------------------|-------|
| id::DbId          | 1     |
| ref_history::DbId | 1     |
| ref_version::DbId | 1     |

 Partner
| KEY               | VALUE |
|-------------------|-------|
| id::DbId          | 2     |
| ref_history::DbId | 4     |
| ref_version::DbId | 4     |


Create tariffs

In [9]:
# create Tariffs
function create_tariff(dsc)
    t = LifeInsuranceDataModel.Tariff()
    tr = LifeInsuranceDataModel.TariffRevision(description=dsc)
    w = Workflow(
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    create_entity!(w)
    create_component!(t, tr, w)
    commit_workflow!(w)
    t.id
end


LifeRiskTariff=create_tariff("Life Risk Insurance")
TerminalIllnessTariff=create_tariff("Terminal Illness")
OccupationalDisability=create_tariff("Occupational Disability")
ProfitParticipationTariff=create_tariff("Profit participation")

find(TariffRevision)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 9 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'9' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 9 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/Searc

┌ Info: UPDATE workflows SET  "id" = E'9', "ref_history" = E'9', "ref_version" = E'9', "tsdb_validfrom" = E'2022-07-08T07:33:49.699+01:00', "tsw_validfrom" = E'2014-05-30T20:00:01.001+00:00', "is_committed" = 1 WHERE workflows.id = '9' RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_history" AS "workflows_ref_history", "workflows"."ref_version" AS "workflows_ref_version", "workflows"."tsdb_validfrom" AS "workflows_tsdb_validfrom", "workflows"."tsw_validfrom" AS "workflows_tsw_validfrom", "workflows"."is_committed" AS "workflows_is_committed" FROM "workflows" WHERE "id" = 9 ORDER BY workflows.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BE

┌ Info: UPDATE workflows SET  "id" = E'10', "ref_history" = E'10', "ref_version" = E'10', "tsdb_validfrom" = E'2022-07-08T07:33:49.736+01:00', "tsw_validfrom" = E'2014-05-30T20:00:01.001+00:00', "is_committed" = 1 WHERE workflows.id = '10' RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_history" AS "workflows_ref_history", "workflows"."ref_version" AS "workflows_ref_version", "workflows"."tsdb_validfrom" AS "workflows_tsdb_validfrom", "workflows"."tsw_validfrom" AS "workflows_tsw_validfrom", "workflows"."is_committed" AS "workflows_is_committed" FROM "workflows" WHERE "id" = 10 ORDER BY workflows.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Inf

┌ Info: UPDATE workflows SET  "id" = E'11', "ref_history" = E'11', "ref_version" = E'11', "tsdb_validfrom" = E'2022-07-08T07:33:49.764+01:00', "tsw_validfrom" = E'2014-05-30T20:00:01.001+00:00', "is_committed" = 1 WHERE workflows.id = '11' RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_history" AS "workflows_ref_history", "workflows"."ref_version" AS "workflows_ref_version", "workflows"."tsdb_validfrom" AS "workflows_tsdb_validfrom", "workflows"."tsw_validfrom" AS "workflows_tsw_validfrom", "workflows"."is_committed" AS "workflows_is_committed" FROM "workflows" WHERE "id" = 11 ORDER BY workflows.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Inf

┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history = E'12' AND tsrdb @> TIMESTAMPTZ E'2038-01-18T03:14:06.999+00:00' AND tsrworld && tstzrange(E'2014-05-31T20:00:01.001+00:00',E'2038-01-19T03:14:06.999+00:00') AND is_committed=1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: UPDATE validityinterval

10-element Vector{TariffRevision}:
 TariffRevision
| KEY                   | VALUE               |
|-----------------------|---------------------|
| description::String   | Life Risk Insurance |
| id::DbId              | 1                   |
| ref_component::DbId   | 1                   |
| ref_invalidfrom::DbId | 9223372036854775807 |
| ref_validfrom::DbId   | 2                   |

 TariffRevision
| KEY                   | VALUE               |
|-----------------------|---------------------|
| description::String   | Terminal Illness    |
| id::DbId              | 2                   |
| ref_component::DbId   | 2                   |
| ref_invalidfrom::DbId | 9223372036854775807 |
| ref_validfrom::DbId   | 3                   |

 TariffRevision
| KEY                   | VALUE               |
|-----------------------|---------------------|
| description::String   | Life Risk Insurance |
| id::DbId              | 3                   |
| ref_component::DbId   | 3                   |
| r

In [11]:
find(Tariff,SQLWhereExpression("id=?",ProfitParticipationTariff))

┌ Info: SELECT "tariffs"."id" AS "tariffs_id", "tariffs"."ref_history" AS "tariffs_ref_history", "tariffs"."ref_version" AS "tariffs_ref_version" FROM "tariffs" WHERE id=E'10' ORDER BY tariffs.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


1-element Vector{Tariff}:
 Tariff
| KEY               | VALUE |
|-------------------|-------|
| id::DbId          | 10    |
| ref_history::DbId | 12    |
| ref_version::DbId | 12    |


In [ ]:


map(find(Tariff)) do t
    SearchLight.delete(t)
end

find(Tariff)

